In [10]:
## SETUP ##

import ccxt
from Values import BYBIT_API_KEY, BYBIT_API_SECRET, OHLCV_DIR
import pandas as pd
import warnings
from pybit import usdt_perpetual
from indicators import *
import importlib
import time

pybit_auth = usdt_perpetual.HTTP(
    endpoint="https://api.bybit.com",
    api_key=BYBIT_API_KEY,
    api_secret=BYBIT_API_SECRET
)

# Switch Logging off and on
debugLog = True

warnings.simplefilter(action='ignore')
pd.set_option('display.max_rows', None)

bybit = ccxt.bybit({'apiKey': BYBIT_API_KEY, 'secret': BYBIT_API_SECRET})
data_dir = OHLCV_DIR


In [11]:
## Test Data ##
base_cur = "USDT"
exch_cur = "DOGE"
symbol = '_'.join([exch_cur, base_cur])
timeframe = "5m"
maxCandles = 500
runUpCandles = 20
# Dataframe from test json file (backtesting)
jsonfile = f"{data_dir}{symbol}-{timeframe}.json"
print(jsonfile)
df = pd.read_json(jsonfile)
df.head()
#df = df[-500:-2]
df.columns = ['timestamp','open','high','low','close','volume']
print(f"type: {type(df['timestamp'][0])}")
#df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
#df['trade'] = 'no_trade'
#trades = pd.DataFrame(columns=['time', 'leverage', 'fees', 'side', 'stake', 'entry_price', 'exit_price', 'percent_difference', 'PNL', 'entry_reason', 'exit_reason', 'entry_balance', 'exit_balance', 'status'])
#superTrend(df, multiplier=5, period=9)
#df_backup = df.copy()
#df.describe()

../../data/bybit/DOGE_USDT-5m.json
type: <class 'numpy.int64'>


,timestamp,open,high,low,close,volume
count,8.568200e+04,85682.000000,85682.000000,85682.000000,85682.000000,8.568200e+04
mean,1.635483e+12,0.212588,0.213050,0.212116,0.212585,9.097324e+05
std,7.420322e+09,0.062386,0.062586,0.062170,0.062383,1.873496e+06
min,1.622630e+12,0.108700,0.109500,0.105800,0.108700,2.000000e+00
25%,1.629056e+12,0.163700,0.164100,0.163325,0.163700,1.539892e+05
50%,1.635483e+12,0.208500,0.208900,0.208000,0.208500,3.815880e+05
75%,1.641909e+12,0.254700,0.255400,0.253900,0.254700,9.365852e+05
max,1.648335e+12,0.442800,0.445000,0.438200,0.442800,7.516842e+07


In [12]:
## FUNCTIONS ##

# Get Balance fetch_balance dict_keys(['info', 'ADA', 'BIT', 'BTC', 'DOT', 'EOS',
#                                      'ETH', 'LTC', 'LUNA', 'MANA', 'SOL', 'USDT',
#                                      'XRP', 'free', 'used', 'total'])
def GetTotalBalance(verbose=debugLog) -> dict:
    full_balances = bybit.fetch_balance()
    totals = full_balances['total']
    balance = {}
    for t in totals:
        if totals[t] > 0:
            balance[str(t)] = totals[t]
    return balance
#print(GetTotalBalance())
def GetFreeBalance(verbose=debugLog) -> dict:
    full_balances = bybit.fetch_balance()
    totals = full_balances['free']
    balance = {}
    for t in totals:
        if totals[t] > 0:
            balance[str(t)] = totals[t]
    return balance
#print(GetFreeBalance())
def GetUsedBalance(verbose=debugLog) -> dict:
    full_balances = bybit.fetch_balance()
    totals = full_balances['used']
    balance = {}
    for t in totals:
        if totals[t] > 0:
            balance[str(t)] = totals[t]
    return balance
#print(GetUsedBalance())

# Get all symbol ids, or all of a given quote currency
def GetSymbols(quote_currency="", verbose=debugLog) -> list:
    marketsFull = bybit.fetch_markets()
    symbols = []
    count = 0
    if quote_currency == "":
        for m in marketsFull:
            symbols.append(m['id'])
    else:
        for m in marketsFull:
            if m['quote'] == quote_currency:
                symbols.append(m['id'])
    if verbose: print(f"count: {len(symbols)}")
    return symbols
#print(GetSymbols('USDT'))

def GetTop24hVol(number: int=0, stripped: bool=True, verbose: bool=debugLog) -> tuple:
    if number > 99:
        if verbose: print("Please pick a smaller number")
        return ()
    tickers = bybit.fetch_tickers()
    all_tickers_sym = tickers.keys()
    vols = {}
    for key in all_tickers_sym:
        tick = tickers[key]
        if  tick['info']['symbol'][-4:] == 'USDT':
            vol = float(tick['quoteVolume']) * tick['last']
            vols[vol] = key
    sorted_vol = sorted(vols.items())
    sorted_vol = list(reversed(sorted_vol))
    if number > 0: sorted_vol = sorted_vol[:number]
    if stripped:
        stripped_list = []
        for res in sorted_vol:
            # BTC/USDT:USDT -> BTCUSDT
            stripped_list.append("".join(res[1].split(':')[0].split('/')))
        sorted_vol = stripped_list
    return tuple(sorted_vol)
#print(GetTop24hVol(20, True, True))

# Create a Simple Buy/Sell Market Order
# amount - size of position in base currency
def CreateSimpleMarketOrder(sym: str = 'ADAUSDT', side: str = 'buy', amount: float = 1.0, verbose=debugLog) -> dict:
    return bybit.create_market_order(sym, side, amount, 0)
#print(CreateSimpleMarketOrder('XRPUSDT'))

# Create a Simple Buy/Sell Limit Order
def CreateSimpleLimitOrder(sym: str = 'ADAUSDT', side: str = 'buy', amount: float = 1.0, price: float = 0.48, verbose=debugLog) -> dict:
    return bybit.create_limit_order(sym, side, amount, price)
#print(CreateSimpleLimitOrder('XRPUSDT', amount=10))

# TODO #
########
def CreatSimpleConditionalOrder():
    return bybit.create_contract_order()

# Get open positions
def GetOpenPositions() -> dict:
    my_positions = pybit_auth.my_position()['result']
    positions = {}
    for p in my_positions:
        if p['data']['size'] > 0: positions[p['data']['symbol']] = p['data']
    return positions
#print(GetOpenPositions())

# Get All active limit orders
def GetActiveOrders(symbol_list: list = []) -> list:
    limits = []
    # default list
    if symbol_list == []: symbol_list =  list(GetTop24hVol(20, True, False))
    for sym in symbol_list:
        my_order = pybit_auth.get_active_order(symbol=sym)['result']['data']
        #print(my_order)
        if my_order != None:
            for ord in my_order:
                if ord['order_status'] != 'Filled' and ord['order_status'] != 'Cancelled': limits.append(ord)
    return limits
#print(GetActiveOrders())

# Get All active conditional orders
def GetConditionalOrders(symbol_list: list = []) -> list:
    conditionals = []
    # default list
    if symbol_list == []: symbol_list = list(GetTop24hVol(20, True, False))
    for sym in symbol_list:
        my_order = pybit_auth.get_conditional_order(symbol=sym)['result']['data']
        #print(my_order)
        if my_order != None:
            for ord in my_order:
                if ord['order_status'] != 'Filled' and ord['order_status'] != 'Deactivated': conditionals.append(ord)
    return conditionals
#print(GetConditionalOrders())

def GetCandles(symbol: str = "DOGEUSDT", timeframe='5min', start: float = 0.0, end: float = 0.0):
    '''Get Candles'''
    return


In [ ]:
# indicator testing
ind = None
t1 = time.time()
ind = RelativeStrengthIndex(df)
t2 = time.time()
print(f"time: {(t2-t1)*1000:.2f} millisecs")
for i in range(len(ind)):
    #print(len(ind[i]))
    print(ind[i])


In [15]:
# Dash Test
import dash
import dash_core_components as dcc
import dash_html_components as html

 


In [ ]:
# init dash app
app = dash.Dash()

colors = {
    'title': ('#ff0000','color'),
    'text': ('#ffffff','color'),
    'plot': ('#aaaaaa','color'),
    'paper': ('#999999','color'),
}

# create tuples for ui dialog name : ('value', type) e.g. 'HL2_color': ("#ffaa00", 'color')
cndl_styles = {
    'rising_fill': ('#33ff33','color'),
    'rising_border': ('#00ff00','color'),
    'rising_border_thickness': ('2px','size'),
    'falling_fill': ('#ff3333','color'),
    'falling_border': ('#ff0000','color'),
    'falling_border_thickness': ('2px','size'),
    'HL2_color': ('#ffaa00','color'),
    'HL2_thickness': ('3px','size')
}

# init layout
app.layout = html.Div([
    html.H1("HelloWorld",
    style = {
        'textAlign': 'center',
        'color': colors['title']
    }),
    html.Div("Dash is pretty cool"),

    dcc.Graph(
        id="TestGraph",
        figure={
            'data':[{'x': [4,6,8], 'y': [12,15,18], 'type': 'bar', 'name': 'First'},
            {'x': [4,6,8], 'y': [10,25,8], 'type': 'bar', 'name': 'Second'}],
            'layout': {
                'title': 'SimpleBarChart',
                'plot_bgcolor': colors['plot'],
                'paper_bgcolor': colors['paper'],
                'font':{
                    'color': colors['text']
                }
            }
        }
    )
])

if __name__ == "__main__":
    app.run_server()